In [1]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots as ms
import plotly.graph_objs as go

In [2]:
deliveries = pd.read_csv("../input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")
matches = pd.read_csv("../input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv")

In [3]:
x = ['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
    'Rising Pune Supergiant', 'Royal Challengers Bangalore',
    'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
    'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
    'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants', 'Delhi Capitals']
y = ['SRH','MI','GL','RPS','RCB','KKR','DC','KXIP','CSK','RR','SRH','KTK','PW','RPS','DC']
matches.replace(x, y, inplace = True)
deliveries.replace(x, y, inplace = True)

Matches in Every Season:

In [4]:
matches["Season"] = matches["date"].str[:4].astype(int)

In [5]:
df1 = matches["Season"].value_counts().rename_axis("Seasons").reset_index(name ="Count")
fig = px.bar(df1, x = "Seasons", y = "Count",template = "plotly_dark",
            title = "Total Matches in Every Season", color_discrete_sequence = ["pink"])
fig.show()

Matches Played, Wins and Win Percentage by each Team.

In [6]:
df_team1 = matches["team1"].value_counts().rename_axis("Team").reset_index(name = "Total Matches")
df_team2 = matches["team2"].value_counts().rename_axis("Team").reset_index(name = "Total Matches")
new = pd.merge(df_team1, df_team2, on = "Team")
new["Total  Matches"] = new["Total Matches_x"]+new["Total Matches_y"] 
total_matches = new[["Team", "Total  Matches"]].sort_values(by ="Total  Matches", ascending = False)
wins = matches["winner"].value_counts().rename_axis("Team").reset_index(name = "Wins")
df = pd.merge(total_matches, wins, on = "Team")
df["Win %"] = round((df["Wins"]/df["Total  Matches"])*100, 1)
mdf = df.melt(id_vars = "Team", value_vars = ["Total  Matches", "Wins", "Win %"],
        var_name = "Teams",value_name = "Count")

In [7]:
fig = px.bar(mdf, x = "Team", y = "Count", 
            color = "Teams", title = "Matches Played, Wins and Win Percentage",
            template = "plotly_dark", barmode = "group")
fig.show()

Venue of Matches:

In [8]:
venues = matches["venue"].value_counts().rename_axis("Venues").reset_index(name = "Count")
venues["size"] =1
fig1 = px.scatter(venues, x = "Venues", y = "Count", template = "plotly_dark",
                  color_discrete_sequence = ["red"],
                  size = "size",
                 size_max = 10)
fig2 = px.line(venues, x = "Venues", y = "Count", template = "plotly_dark", color_discrete_sequence = ["cyan"])
fig = go.Figure(fig1["data"]+fig2["data"])
fig.update_layout(template = "plotly_dark", height = 800,
                 title = "Venues VS Matches")
fig.show()

Number of matches each umpire occured  

In [9]:
u1 = matches["umpire1"].value_counts().rename_axis("Umpire").reset_index(name = "Counts")
u2 = matches["umpire2"].value_counts().rename_axis("Umpire").reset_index(name = "Counts")
mumpire = pd.merge(u1, u2, on = "Umpire")
mumpire["Counts "] = mumpire["Counts_x"]+mumpire["Counts_y"]
umpires = mumpire[["Umpire",  "Counts "]].sort_values(by = "Counts ", ascending = False)

In [10]:
fig = px.bar(umpires, x = "Umpire", y = "Counts ", template = "plotly_dark",
            title = "Umpires in Matches",color_discrete_sequence = ["lightblue"])
fig.show()

Toss Decision

In [11]:
field = matches[matches["toss_decision"] == "field"]["toss_decision"].value_counts()
bat = matches[matches["toss_decision"] == "bat"]["toss_decision"].value_counts()
toss = pd.DataFrame({"bat":list(bat), "field":list(field)})
mtoss = toss.melt(value_vars = ["bat", "field"], var_name = "toss_decision", value_name = "Count")

In [12]:
fig =px.bar(mtoss, x = "toss_decision", y = "Count",
           template = "plotly_dark",color_discrete_sequence = ["blue"])
fig.show()

Total and Average runs each Season

In [13]:
mergedf = pd.merge(matches, deliveries, on = "id")
Season1 = mergedf.pivot_table(index = "Season", values = "total_runs",aggfunc = "sum").reset_index()
Season2 = mergedf.pivot_table(index = ["Season", "id"], values = "total_runs", aggfunc = "sum").reset_index()
dff = Season2["Season"].value_counts().rename_axis("Season").reset_index(name = "total matches").sort_values(by = "Season")  
newdf = pd.merge(Season1, dff, on ="Season")
newdf["average runs"] = newdf["total_runs"]/newdf["total matches"]

In [14]:
fig1 = px.line(newdf, x = "Season", y = "total_runs", color_discrete_sequence = ["red"],
            template = "plotly_dark")
fig2 = px.scatter(newdf, x = "Season", y = "total_runs", color_discrete_sequence = ["red"],
            template = "plotly_dark")
fig = go.Figure(fig1["data"]+fig2["data"])
fig.update_layout(template = "plotly_dark", title = "Total runs per Season")
fig.update_xaxes(title = "year")
fig.update_yaxes(title = "Total Runs")
fig.show()
fig11 = px.line(newdf, x = "Season", y = "average runs", color_discrete_sequence = ["lightblue"],
            template = "plotly_dark")
fig22 = px.scatter(newdf, x = "Season", y = "average runs", color_discrete_sequence = ["lightblue"],
            template = "plotly_dark")
figg = go.Figure(fig11["data"]+fig22["data"])
figg.update_layout(template = "plotly_dark", title = "Average runs per Season")
figg.update_xaxes(title = "year")
figg.update_yaxes(title = "Average Runs")
figg.show()

Run Distribution Over Years

In [15]:
df1 = mergedf[mergedf["batsman_runs"] == 6].pivot_table(index = ["Season", "id"], values = "batsman_runs",
                                                 aggfunc = "sum").reset_index().pivot_table(index = ["Season"], 
                                                values = "batsman_runs", aggfunc = "sum").reset_index()
df2 = mergedf[mergedf["batsman_runs"] == 4].pivot_table(index = ["Season", "id"], values = "batsman_runs",
                                                 aggfunc = "sum").reset_index().pivot_table(index = ["Season"], 
                                                values = "batsman_runs", aggfunc = "sum").reset_index()
df3 = mergedf.pivot_table(index = ["Season", "id"], values = "total_runs",
                                                 aggfunc = "sum").reset_index().pivot_table(index = ["Season"], 
                                                values = "total_runs", aggfunc = "sum").reset_index()
new = pd.merge(df1, df2, on= "Season").merge(df3, on = "Season")
new["remaining runs"] = new["total_runs"]-(new["batsman_runs_x"]+new["batsman_runs_y"])
new.rename(columns = {
    "batsman_runs_x":"Run by 6's",
    "batsman_runs_y":"Run by 4's"
}, inplace = True)
mnew = new.melt(id_vars = "Season", value_vars = ["Run by 6's", "Run by 4's", "remaining runs"],
        var_name = "Cases", value_name = "Run Distribution")


In [16]:
fig = px.bar(mnew,color = "Cases",x = "Season", y = "Run Distribution",
            template = "plotly_dark", barmode = "group", title = "Run Distribution per year")
fig.show()

Number of Times a team has scored moe than 200

In [17]:
df1 = mergedf[mergedf["inning"] == 1].pivot_table(index = ["inning", "id", "batting_team"], values = "total_runs", aggfunc = "sum").reset_index()
df2 = mergedf[mergedf["inning"] == 2].pivot_table(index = ["inning", "id", "batting_team"], values = "total_runs",aggfunc = "sum").reset_index()
new = pd.merge(df1, df2, on = "id")
op1 = new[new["total_runs_x"]> 200]["batting_team_x"].value_counts().rename_axis("Batting Team").reset_index(name = "Total Times1")
op2 = new[new["total_runs_y"]>200]["batting_team_y"].value_counts().rename_axis("Batting Team").reset_index(name = "Total Times2")
j = pd.merge(op1, op2, on = "Batting Team")
j["Total Times"] = j["Total Times1"]+j["Total Times2"]
jj = j[["Batting Team", "Total Times"]]

In [18]:
fig = px.bar(jj, x = "Batting Team", y = "Total Times", template = "plotly_dark",
             color_discrete_sequence = ["cyan"],
            title = "No of Times a team has scored more than 200")
fig.show()

Is it possible to chase 200+ Targets?

In [19]:
high = new[(new["total_runs_x"] > 200) | (new["total_runs_y"] > 200)]
high2 = high[high["total_runs_y"] > high["total_runs_x"]]
a = high2["total_runs_y"].value_counts().sum()
high3 = high[high["total_runs_x"] > high["total_runs_y"]]
b = high3["total_runs_y"].value_counts().sum()
df = pd.DataFrame({"Yes":[a], "No":[b]}).melt(value_vars = ["Yes", "No"],
                                        var_name = "Cases", value_name = "Count")

In [20]:
fig = px.pie(df, names = "Cases", values = "Count", template = "plotly_dark",
            title = "200 score Chased?")
fig.show()

Runs Per over of Each Team

In [21]:
over = mergedf.pivot_table(index = ["batting_team","over"], values = "total_runs", aggfunc = "sum").reset_index()
over2 = over[(over["batting_team"]  != "KTK") & (over["batting_team"]  != "GL") & (over["batting_team"]  != "PW") & (over["batting_team"]  != "RPS")]    


In [22]:
fig = px.line(over2, x = "over",
             y = "total_runs", color = "batting_team",template = "plotly_dark",
             title = "Average Run in Each Over")
fig.show()

Runs in powerplay of Each Match

In [23]:
n1 = mergedf[mergedf["inning"] == 1]
n2 = n1.pivot_table(index = ["id", "over"], values = "total_runs", aggfunc = "sum").reset_index()
n3 = n2[n2["over"] < 6].pivot_table(index = "id", values = ["total_runs", "over"], aggfunc= "sum").reset_index()
b = []
for i in range(1,817):
    b.append(i)
n4 = pd.DataFrame({"matches":b})
n5 = n3.join(n4)
n5
n11 = mergedf[mergedf["inning"] == 2]
n22 = n11.pivot_table(index = ["id", "over"], values = "total_runs", aggfunc = "sum").reset_index()
n33 = n22[n22["over"] < 6].pivot_table(index = "id", values = ["total_runs", "over"], aggfunc= "sum").reset_index() 
c = []
for i in range(1,815):
    c.append(i)
n44= pd.DataFrame({"matches":b})
n55 = n33.join(n44)

In [24]:
fig = px.line(n5, x = "matches", 
             y = "total_runs", 
             template = "plotly_dark",
             color_discrete_sequence = ["cyan"],
             title = "Inning 1")
fig.show()
figg = px.line(n55, x = "matches", 
             y = "total_runs", 
             template = "plotly_dark",
             color_discrete_sequence = ["red"],
              title = "Inning 2")
figg.show()

Powerplay Average Runs

In [25]:
a1 = mergedf[mergedf["inning"] == 1]
a2 = a1.pivot_table(index = ["Season","id", "over"], values = ["total_runs"], aggfunc = "sum").reset_index()
a3 = a2[a2["over"] < 6].pivot_table(index = ["Season", "id"], values = "total_runs", aggfunc = "sum").reset_index()
a4 = a3.pivot_table(index = "Season", values = ["total_runs"], aggfunc = "mean").reset_index()
a11 = mergedf[mergedf["inning"] == 2]
a22 = a1.pivot_table(index = ["Season","id", "over"], values = ["total_runs"], aggfunc = "sum").reset_index()
a33 = a22[a22["over"] < 6].pivot_table(index = ["Season", "id"], values = "total_runs", aggfunc = "sum").reset_index()
a44 = a33.pivot_table(index = "Season", values = ["total_runs"], aggfunc = "mean").reset_index()
new = pd.merge(a4,a4, on = "Season")
new.rename(columns = {"total_runs_x":"Inning1 runs",
                     "total_runs_y":"Inning2 runs"}, inplace = True)
mnew = new.melt(id_vars = "Season", value_vars = ["Inning1 runs", "Inning2 runs"],
        var_name = "Cases", value_name = "Run")

In [26]:
fig = px.bar(mnew, x= "Season", y = "Run", color = "Cases",
            barmode = "group", template = "plotly_dark")
fig.show()

Top 20 Batsman

In [27]:
a1 = mergedf.pivot_table(index = "batsman", values = ["batsman_runs"], aggfunc = "sum").reset_index().sort_values(by = "batsman_runs",ascending = False)
a2 = deliveries[["batsman", "ball"]]
a23 = a2["batsman"].value_counts().rename_axis("batsman").reset_index(name = "balls")
df1 = pd.merge(a1, a23, on = "batsman")
df1["strike rate"] = df1["batsman_runs"]/df1["balls"]*100
a3 = deliveries.pivot_table(index = ["batsman", "id"], values = "total_runs", aggfunc = "sum").reset_index()
a4 = a3["batsman"].value_counts().rename_axis("batsman").reset_index(name = "total matches")
df3 = pd.merge(df1, a4, on = "batsman")
df3["average score"] = df3["batsman_runs"]/df3["total matches"]
a5 = mergedf[mergedf["batsman_runs"] == 6].pivot_table(index = "batsman", values = "total_runs", aggfunc = "sum").reset_index()
a5["sixes"] = a5["total_runs"]//6
a55 = a5[["batsman", "sixes"]]
a6 = mergedf[mergedf["batsman_runs"] == 4].pivot_table(index = "batsman", values = "total_runs", aggfunc = "sum").reset_index()
a6["fours"] = a6["total_runs"]//4
a66 = a6[["batsman", "fours"]]
result = pd.merge(df3, a55, on = "batsman").merge(a66, on = "batsman").head(20)

In [28]:
fig = px.scatter(result, x = "batsman", y = "total matches", size = "sixes",
                 color = "sixes",
                template = "plotly_dark",
                title = "Batsman analysis",
                size_max = 20)
fig.show()

Orange-Cap Holders

In [29]:
df1 = mergedf.pivot_table(index = ["Season", "batsman"], values = "batsman_runs", aggfunc = "sum").reset_index().sort_values(by = "batsman_runs", ascending = False)
orange = df1.drop_duplicates(subset = ["Season"], keep = "first").sort_values(by = "Season")

In [30]:
fig = px.bar(orange, x = "Season", y = "batsman_runs",
            hover_name = "batsman",
            template = "plotly_dark",
            color_discrete_sequence = ["orange"],
            title = "Orange-Cap Holders")
fig.show()

Purple-Cap Holders

In [31]:
df1 = mergedf[mergedf["dismissal_kind"] != "run out"].pivot_table(
    index = ["Season", "bowler"],values = ["is_wicket"], aggfunc = "sum").reset_index().sort_values(
by = "is_wicket", ascending = False).drop_duplicates(subset = ["Season"], keep = "first").sort_values(
by = "Season")

In [32]:
fig = px.bar(df1, x = "Season", y = "is_wicket",
            color_discrete_sequence = ["#aa23ff"],
            template = "plotly_dark", title = "Purple-Cap Holders")
fig.show()

Top 10 Bowlers in Ipl History

In [33]:
df = mergedf[mergedf["dismissal_kind"] != "run out"].pivot_table(index = ["bowler"], values = ["is_wicket"],
        aggfunc = "sum").reset_index().sort_values(by = "is_wicket", ascending = False).head(10)

In [34]:
fig1 = px.line(df, x = "bowler", y = "is_wicket",
             template = "plotly_dark", title = "Top 10 bowlers in IPL",
             color_discrete_sequence = ["red"])
fig2 = px.scatter(df, x = "bowler", y = "is_wicket",
             template = "plotly_dark", title = "Top 10 bowlers in IPL",
             color_discrete_sequence = ["red"])
fig = go.Figure(fig1["data"]+fig2["data"])
fig.update_layout(template = "plotly_dark", title = "Top 10 bowlers in IPL history")
fig.update_xaxes(title = "Wickets Taken")
fig.update_yaxes(title = "Top 10 Bowlers")
fig.show()

Which team win highest number of seasons

In [35]:
wins = matches.drop_duplicates(subset = ["Season"],keep = "last")[["winner"]].value_counts().rename_axis("Team").reset_index(
name = "No of Wins")

In [36]:
fig = px.pie(wins, names = "Team", values = "No of Wins", template = "plotly_dark",
            title = "Winner of Ipl season")
fig.show()

Matches Played vs wins in Final Match of Every Season

In [37]:
df1 = matches.drop_duplicates(subset = ["Season"],keep = "last")
d1 = df1[["team1"]].value_counts().rename_axis("Team").reset_index(name = "Played")
d2 = df1[["team2"]].value_counts().rename_axis("Team").reset_index(name = "Played")
d3 = df1[["winner"]].value_counts().rename_axis("Team").reset_index(name = "Winner")
df2 = pd.merge(d1, d2, on ="Team", how = "outer").merge(d3, on = "Team", how = "outer").fillna(0)
df2["Total Matches"] = df2["Played_x"]+df2["Played_y"]
result = df2[["Team", "Winner", "Total Matches"]].melt(id_vars = "Team", value_vars = ["Winner", "Total Matches"],
                                             var_name = "Cases", value_name = "Count")

In [38]:
fig = px.bar(result, x = "Team", y = "Count",
            color = "Cases", template = "plotly_dark", barmode = "group")
fig.show()

Is Toss Decision is Final Decison in Final?

In [39]:
df1 = matches.drop_duplicates(subset = ["Season"],keep = "last")
w1 = df1[df1["toss_winner"] == df1["winner"]][["winner"]].value_counts().sum()
w2 = df1[df1["toss_winner"] != df1["winner"]][["winner"]].value_counts().sum()
result = pd.DataFrame({"yes":[w1], "no":[w2]}).melt(value_vars = ["yes", "no"],
                                          value_name = "Count", var_name = "Cases")

In [40]:
fig = px.pie(result, names = "Cases", values = "Count", template  = "plotly_dark",
            title = "Winner of IPL season")
fig.show()

Top 10 batsman in each Category

In [41]:
df1 = mergedf[mergedf["batsman_runs"] == 6].pivot_table(index = "batsman", values = "batsman_runs",
                        aggfunc = "sum").reset_index().sort_values(by = "batsman_runs", ascending = False).head(10)  
df1["sixes"] = df1["batsman_runs"]//6
df2 = mergedf[mergedf["batsman_runs"] == 4].pivot_table(index = "batsman", values = "batsman_runs",
                        aggfunc = "sum").reset_index().sort_values(by = "batsman_runs", ascending = False).head(10)  
df2["fours"] = df2["batsman_runs"]//4
df3 = mergedf[mergedf["batsman_runs"] == 2].pivot_table(index = "batsman", values = "batsman_runs",
                        aggfunc = "sum").reset_index().sort_values(by = "batsman_runs", ascending = False).head(10)  
df3["twos"] = df3["batsman_runs"]//2
df4 = mergedf[mergedf["batsman_runs"] == 1].pivot_table(index = "batsman", values = "batsman_runs",
                        aggfunc = "sum").reset_index().sort_values(by = "batsman_runs", ascending = False).head(10)  
df4["ones"] = df4["batsman_runs"]//1

In [42]:
fig1 = px.line(df1, x = "batsman", y = "sixes", color_discrete_sequence = ["lightblue"])
fig2 = px.line(df2, x = "batsman", y = "fours",color_discrete_sequence = ["yellow"])
fig3 = px.line(df3, x = "batsman", y = "twos",color_discrete_sequence = ["cyan"])
fig4 = px.line(df4, x = "batsman", y = "ones",color_discrete_sequence = ["red"])
trace1 = fig1["data"][0]
trace2 = fig2["data"][0]
trace3 = fig3["data"][0]
trace4 = fig4["data"][0]
fig = ms(rows = 4, cols = 1, subplot_titles = ["Top 6's Scorer",
                                              "Top 4's Scorer",
                                              "Top 2's Scorer",
                                              "Top 1's Scorer"])
fig.add_trace(trace1, row = 1, col = 1)
fig.add_trace(trace2, row = 2, col =1)
fig.add_trace(trace3, row = 3, col = 1)
fig.add_trace(trace4, row = 4, col = 1)
fig.update_layout(title = "Top Scorers in each Category", template = "plotly_dark")
fig.show()